In [52]:
import turicreate

In [53]:
sf=turicreate.SFrame('people_wiki.sframe')

In [54]:
obama=sf[sf['name']=="Barack Obama"]

In [55]:
obama['words']=turicreate.text_analytics.count_words(obama['text'])

In [56]:
obama

URI,name,text,words
<http://dbpedia.org/resource/Barack_Obama> ...,Barack Obama,barack hussein obama iibrk husen bm born august ...,"{'normalize': 1.0,'sought': 1.0, 'combat': ..."


In [57]:
obamawordtable=obama[['words']].stack('words',new_column_name=['word','frequency'])
#Important Method for making a table of words

In [58]:
obamawordtable 

word,frequency
normalize,1.0
sought,1.0
combat,1.0
continued,1.0
unconstitutional,1.0
8,1.0
californias,1.0
1996,1.0
marriage,1.0
defense,1.0


In [59]:
#Sort table by count
obamawordtable.sort('frequency',ascending=False)

word,frequency
the,40.0
in,30.0
and,21.0
of,18.0
to,14.0
his,11.0
obama,9.0
act,8.0
a,7.0
he,7.0


## Computing TFIDF of All

In [60]:
sf['word']=turicreate.text_analytics.count_words(sf['text'])

In [61]:
sf['tfidf']=turicreate.text_analytics.tf_idf(sf['text'])

In [62]:
obama=sf[sf['name']=='Barack Obama']

In [63]:
obamatf=obama[['tfidf']].stack('tfidf',new_column_name=['word','count']).sort('count',ascending=False)

In [64]:
obamatf

word,count
obama,43.2956530720749
act,27.67822262297991
iraq,17.747378587965535
control,14.887060845181308
law,14.722935761763422
ordered,14.533373950913514
military,13.115932778499415
involvement,12.784385241175055
response,12.784385241175055
democratic,12.410688697332166


## Distance Metric

In [65]:
clinton=sf[sf['name']=="Bill Clinton"]

In [66]:
beckham=sf[sf['name']=="David Beckham"]

In [67]:
turicreate.distances.cosine(obama['tfidf'][0],clinton['tfidf'][0])

0.8339854936884277

In [68]:
turicreate.distances.cosine(obama['tfidf'][0],beckham['tfidf'][0])

0.9791305844747478

In [69]:
turicreate.distances.cosine(beckham['tfidf'][0],clinton['tfidf'][0])

0.9693008710082479

# Nearest Neighbour

In [70]:
knnmodel=turicreate.nearest_neighbors.create(sf,features=['tfidf'],label='name')
#create a model of nearest neighbour of sf using tfidf feature and return the name

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [71]:
knnmodel.query(obama)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 16.144ms     |

| Done         |         | 100         | 298.87ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Barack Obama,0.0,1
0,Joe Biden,0.7941176470588236,2
0,Joe Lieberman,0.7946859903381642,3
0,Kelly Ayotte,0.8119891008174387,4
0,Bill Clinton,0.8138528138528138,5


In [72]:
knnmodel.query(beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 26.605ms     |

| Done         |         | 100         | 333.884ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,David Beckham,0.0,1
0,Steven Gerrard,0.7416107382550335,2
0,Didier Drogba,0.7477477477477478,3
0,Gordon Strachan,0.75,4
0,Wayne Rooney,0.7523510971786834,5


In [73]:
knnmodel.query(clinton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 17.868ms     |

| Done         |         | 100         | 352.111ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Bill Clinton,0.0,1
0,Hillary Rodham Clinton,0.7931034482758621,2
0,George W. Bush,0.7944325481798715,3
0,Dick Cheney,0.8064516129032258,4
0,Barack Obama,0.8138528138528138,5


In [77]:
model=turicreate.nearest_neighbors.create(sf,features=['word'],label='name')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [78]:
model.query(obama)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 8.242ms      |

| Done         |         | 100         | 202.477ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Barack Obama,0.0,1
0,Joe Biden,0.7941176470588236,2
0,Joe Lieberman,0.7946859903381642,3
0,Kelly Ayotte,0.8119891008174387,4
0,Bill Clinton,0.8138528138528138,5
